In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from dotenv import load_dotenv
from datetime import datetime
import os
import time
import random
import pyperclip
from selenium.webdriver.common.keys import Keys

In [ ]:
def load_contacts_from_csv(file_path):
    """Carica il CSV dei contatti in un DataFrame."""
    try:
        df = pd.read_csv(file_path)
        print(f"Caricati {len(df)} contatti.")
        return df
    except FileNotFoundError:
        print(f"File non trovato: {file_path}")
        return pd.DataFrame()

def filter_contacts(df):
    """Filtra i contatti in base alla città o regione."""
    filter_type = input("Vuoi filtrare per città o regione? (città/regione/nessuno): ").strip().lower()

    if filter_type == 'città':
        city = input("Inserisci la città per filtrare: ").strip()
        filtered_df = df[df['Città'].str.contains(city, case=False, na=False)]
    elif filter_type == 'regione':
        region = input("Inserisci la regione per filtrare: ").strip()
        filtered_df = df[df['Regione'].str.contains(region, case=False, na=False)]
    else:
        filtered_df = df

    print(f"Filtrati {len(filtered_df)} contatti.")
    return filtered_df

def contact_filter(df):
    """Filtra i contatti in base allo stato di contatto."""
    contact_choice = input("Vuoi contattare tutti, già contattati o non contattati? (tutti/già/non): ").strip().lower()

    if contact_choice == 'già':
        filtered_df = df[df['Contattato'].notna()]  # Filtra quelli già contattati (colonna non vuota)
    elif contact_choice == 'non':
        filtered_df = df[df['Contattato'].isna()]  # Filtra quelli mai contattati (colonna vuota)
    else:
        filtered_df = df  # Contatta tutti

    print(f"Contatti da processare: {len(filtered_df)}")
    return filtered_df


def select_random_contacts(df):
    """Chiede quanti contatti prendere casualmente e restituisce un subset del DataFrame."""
    num_contacts = input("Quanti contatti vuoi prendere? (inserisci un numero o scrivi 'tutti' per contattare tutti): ").strip().lower()

    if num_contacts == 'tutti':
        selected_df = df
    else:
        try:
            num_contacts = int(num_contacts)
            selected_df = df.sample(n=num_contacts)  # Prende casualmente n contatti
        except ValueError:
            print("Input non valido, prendo tutti i contatti.")
            selected_df = df

    print(f"Selezionati {len(selected_df)} contatti.")
    return selected_df


def display_and_confirm_contacts(df):
    """Mostra i contatti filtrati e chiede conferma o rimozione di contatti."""
    print("\n--- Lista contatti target ---")
    print(df[['ProfileLink', 'Città', 'Regione']].to_string(index=False))

    confirm = input("\nVuoi procedere con questi contatti? (sì/no): ").strip().lower()
    
    if confirm == 'no':
        links_to_remove = input("Inserisci i link dei profili da rimuovere separati da una virgola: ").strip().split(',')
        links_to_remove = [link.strip() for link in links_to_remove]
        
        # Rimuovi le righe con i link specificati
        df = df[~df['ProfileLink'].isin(links_to_remove)]
        print(f"{len(links_to_remove)} contatti rimossi. Rimangono {len(df)} contatti.")
    
    return df

def update_contact_status(original_df, filtered_df):
    """Aggiorna lo stato di contatto nel DataFrame originale."""
    for index, row in filtered_df.iterrows():
        original_df.at[index, 'Contattato'] = 'y'
        original_df.at[index, 'Data Contatto'] = datetime.now().strftime('%d-%m-%Y')
        original_df.at[index, 'Owner'] = 'Automa'

def save_contacts_to_csv(df, file_path):
    """Salva il DataFrame completo aggiornato in un file CSV."""
    df.to_csv(file_path, index=False)
    print(f"Contatti aggiornati salvati in {file_path}.")

def generate_dynamic_message(full_name, group_name):
    """Genera il messaggio personalizzato con il nome e il gruppo."""
    
    message_template = "Ciao, ho visto che fai parte del gruppo {group_name}! 😊 Come va? Volevo dirti che il 13 e il 14 dicembre stiamo organizzando due cene gioco a Torino, e sarebbe fantastico averti con noi! 🎲 Sono serate pensate per divertirci insieme: mangeremo, berremo e giocheremo con i prototipi dei Game Designer della nostra community BGSC, ma ognuno può portare i propri giochi (ovviamente). È un’occasione unica per provare giochi nuovi e passare del tempo in compagnia e giocando! Trovi tutte le informazioni su luogo e dettagli qui:https://boardgamesocialclub.org/location/BarBillar   Ah, e se vuoi dare un’occhiata ai prototipi in gara al XMAS, li trovi qui: https://boardgamesocialclub.org/game-designer-prototipi  Spero di vederti presto, un abbraccio! "
    
    # Inserisci il nome e il gruppo nel messaggio
    return message_template.format(group_name=group_name)


def send_message_to_profile(driver, profile_url, message):
    """Apre la pagina del profilo, invia il messaggio e gestisce eventuali errori."""
    try:
        # Apri il profilo
        driver.get(profile_url)
        
        # Inserisci un'attesa casuale tra 12 e 58 secondi (convertiti in secondi)
        random_wait = random.randint(12, 120)
        print(f"Attesa di {random_wait } secondi prima di inviare il messaggio...")
        time.sleep(random_wait)

        # Trova e clicca sul pulsante aggiungi amico
        try:
            amico_btn = driver.find_element(By.XPATH, '//*[@aria-label="Aggiungi agli amici"]')
            amico_btn.click()
            time.sleep(5)  # Attendi che il menu del messaggio si apra
        except:
            print(f"Non è stato possibile aggiungere agli amici {profile_url}")

        # Trova e clicca sull'icona del messaggio
        message_icon = driver.find_element(By.XPATH, '//*[@aria-label="Messaggio"]')
        message_icon.click()
        time.sleep(5)  # Attendi che il menu del messaggio si apra

        # Inserisci il messaggio e invialo
        message_box = driver.find_element(By.XPATH, '//*[@aria-placeholder="Aa"]')  # XPath del campo messaggio
        message_box.click()
        time.sleep(5)
        #message_box.send_keys(message)
        pyperclip.copy(message)
        message_box.send_keys(Keys.CONTROL, 'v')  # Su MacOS, usa Keys.COMMAND
        time.sleep(7)
        
        send_button = driver.find_element(By.XPATH, '//*[@aria-label="Premi Invio per inviare"]')  # Bottone invia (può variare)
        send_button.click()
        time.sleep(3)  # Attendi che il messaggio venga inviato

        print(f"Messaggio inviato a {profile_url}")
        
        close_button = driver.find_element(By.XPATH, '//*[@aria-label="Chiudi chat"]')  # Bottone invia (può variare)
        close_button.click()
        time.sleep(3)
        
        return True
    except (NoSuchElementException, WebDriverException) as e:
        print(f"Errore nell'inviare il messaggio a {profile_url}: {e}")
        return False

def update_contact_status(original_df, filtered_df):
    """Aggiorna lo stato di contatto nel DataFrame originale usando gli indici corretti."""
    for index in filtered_df.index:
        original_df.at[index, 'Contattato'] = 'y'
        original_df.at[index, 'Data Contatto'] = datetime.now().strftime('%d-%m-%Y')
        original_df.at[index, 'Owner'] = 'Automa'
        
def login_to_facebook(driver, username, password):
    """Esegue il login a Facebook."""
    driver.get('https://www.facebook.com/login')
    time.sleep(10)  # Attendi il caricamento della pagina

    email_input = driver.find_element(By.XPATH, '//input[@name="email"]')
    email_input.send_keys(username)

    password_input = driver.find_element(By.XPATH, '//*[@id="pass"]')
    password_input.send_keys(password)

    login_button = driver.find_element(By.XPATH, '//*[@id="loginbutton"]')
    login_button.click()
    time.sleep(8)  # Attendi l'accesso

def main():
    username = ''
    password = ''

    chrome_options = Options()
    chrome_options.add_argument('--no-default-browser-check')
    chrome_options.add_argument('--disable-default-apps')
    chrome_options.add_argument('--disable-infobars')
    chrome_options.add_argument('--disable-popup-blocking')
    chrome_options.add_argument("--disable-search-engine-choice-screen")

    driver_path = r'chromedriver-137.exe'
    service = ChromeService(driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    login_to_facebook(driver, username, password)
    print("Login eseguito con successo.")

    file_path = 'contatti_torino.csv'
    
    contacts_df = load_contacts_from_csv(file_path)

    if contacts_df.empty:
        print("Nessun contatto trovato, esco dal programma.")
        driver.quit()
        return

    filtered_df = filter_contacts(contacts_df)
    filtered_df = contact_filter(filtered_df)
    filtered_df = select_random_contacts(filtered_df)
    filtered_df = display_and_confirm_contacts(filtered_df)

    for index, row in filtered_df.iterrows():
        full_name = row['Full Name']
        group_name = row['Gruppo']
        profile_url = row['ProfileLink']
        message = generate_dynamic_message(full_name, group_name)
        
        success = send_message_to_profile(driver, profile_url, message)
        if success:
            update_contact_status(contacts_df, filtered_df)

    save_contacts_to_csv(contacts_df, file_path)
    driver.quit()

if __name__ == '__main__':
    main()